In [76]:
import pandas as pd
df=pd.read_csv("USA_Housing.csv")
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [77]:
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA


In [78]:
x = df.drop("Price" , axis=1).values
y = df["Price"].values.reshape(-1,1)

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

kf=KFold(n_splits=5 ,shuffle=True , random_state=42)
beta_score = None
r2_score = np.inf
betas = []



In [79]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Features and target
x = df.drop("Price", axis=1).values
y = df["Price"].values.reshape(-1, 1)

# Scale features
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# K-Fold setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_beta = None
best_r2 = -np.inf   
betas = []
scores = []

for fold, (train_idx, test_idx) in enumerate(kf.split(x_scaled), 1):
    X_train, X_test = x_scaled[train_idx], x_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    
    X_train_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]


    beta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ (X_train_bias.T @ y_train)

    
    y_pred = X_test_bias @ beta

    
    score = r2_score(y_test, y_pred)
    scores.append(score)
    betas.append(beta)

    print(f"Fold {fold} - R² Score: {score:.4f}")

    
    if score > best_r2:
        best_r2 = score
        best_beta = beta

print("\nBest R² Score:", best_r2)
print("Best β Coefficients:\n", best_beta.flatten())


Fold 1 - R² Score: 0.9180
Fold 2 - R² Score: 0.9146
Fold 3 - R² Score: 0.9116
Fold 4 - R² Score: 0.9193
Fold 5 - R² Score: 0.9244

Best R² Score: 0.9243869413350317
Best β Coefficients:
 [1.23161736e+06 2.30225051e+05 1.63956839e+05 1.21115120e+05
 7.83467170e+02 1.50662447e+05]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Load the dataset (assuming it's already downloaded from the previous step)
df = pd.read_csv("USA_housing.csv")

# Separate features (X) and target (Y)
X = df.drop('Price', axis=1)
Y = df['Price']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data: 56% train, 14% validation, 30% test
# First, split into training (56%) and temp (44%)
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X_scaled, Y, test_size=0.44, random_state=42
)
# Then, split the temp set into validation (14%) and test (30%)
X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp, Y_temp, test_size=(30/44), random_state=42
)

# Add intercept term to all sets
X_train_b = np.c_[np.ones(X_train.shape[0]), X_train]
X_val_b = np.c_[np.ones(X_val.shape[0]), X_val]
X_test_b = np.c_[np.ones(X_test.shape[0]), X_test]

# Gradient Descent function
def gradient_descent(X, y, learning_rate, n_iterations):
    m = X.shape[0]
    beta = np.random.randn(X.shape[1])  # Initialize beta randomly
    for iteration in range(n_iterations):
        gradients = (2/m) * X.T @ (X @ beta - y)
        beta = beta - learning_rate * gradients
    return beta

learning_rates = [0.001, 0.01, 0.1, 1]
best_beta = None
best_r2_val = -np.inf

print("--- Training with Gradient Descent for different learning rates ---")
for lr in learning_rates:
    print(f"\nTraining with learning rate: {lr}")
    beta_gd = gradient_descent(X_train_b, Y_train.values, lr, 1000)

    # Compute R2 score for validation set
    y_val_pred = X_val_b @ beta_gd
    r2_val = r2_score(Y_val, y_val_pred)
    print(f"R2 score on validation set: {r2_val:.4f}")

    # Check if this beta is the best so far
    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_beta = beta_gd

print("\n--- Final Evaluation ---")
print("Best Regression Coefficients (beta) based on validation set:", best_beta)

# Final evaluation on the test set with the best coefficients
y_test_pred = X_test_b @ best_beta
r2_test = r2_score(Y_test, y_test_pred)
print(f"R2 score on test set with best coefficients: {r2_test:.4f}")

In [80]:
columns = [
    "symboling", "normalized_losses", "make", "fuel_type", "aspiration",
    "num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base",
    "length", "width", "height", "curb_weight", "engine_type", "num_cylinders",
    "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
    "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"
]

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(url, names=columns ,na_values ='?')
df.head()




,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [81]:
# df = df.mask(df =='?' ,np.nan)

In [66]:
# Drop rows where price is NaN
df.dropna(subset=['price'], inplace=True)

# Convert price to numeric
df['price'] = pd.to_numeric(df['price'])

# Impute missing values with mean (for numeric columns)
for col in df.columns:
    if df[col].dtype in ['float64', 'int64']:
        df[col].fillna(df[col].mean(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)


/var/folders/b1/956l7sdx4mv8xlc1mbdvmy0c0000gn/T/ipykernel_40651/272903948.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/var/folders/b1/956l7sdx4mv8xlc1mbdvmy0c0000gn/T/ipykernel_40651/272903948.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

In [67]:
word_to_num = {
    'two': 2, 'three': 3, 'four': 4, 'five': 5,
    'six': 6, 'eight': 8, 'twelve': 12
}

df['num_doors'] = df['num_doors'].map(word_to_num)
df['num_cylinders'] = df['num_cylinders'].map(word_to_num)

In [68]:
df = pd.get_dummies(df, columns=['body_style', 'drive_wheels'], drop_first=True)

In [69]:
from sklearn.preprocessing import LabelEncoder

for col in ['make', 'aspiration', 'engine_location', 'fuel_type']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [70]:
df['fuel_system'] = df['fuel_system'].apply(lambda x: 1 if 'pfi' in x else 0)

In [72]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Separate features and target
X = df.drop("price", axis=1)
Y = df["price"]

# Encode categorical columns
for col in X.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Now scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [73]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train, Y_train)
Y_pred = lr.predict(X_test)

print("R² Score (Original Features):", r2_score(Y_test, Y_pred))

R² Score (Original Features): 0.8681988849804585


In [74]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # retain 95% variance
X_pca = pca.fit_transform(X_scaled)

X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(X_pca, Y, test_size=0.3, random_state=42)

lr_pca = LinearRegression()
lr_pca.fit(X_train_pca, Y_train_pca)
Y_pred_pca = lr_pca.predict(X_test_pca)

print("R² Score (PCA Features):", r2_score(Y_test_pca, Y_pred_pca))

R² Score (PCA Features): 0.85936191010145
